In [73]:
#are there other data that is not here
#is it correlated/caused    #target leakage -> time leakage , hving meds predict you sick. but having meds comes after,hence cant use a predictor
#do scatterplot of y with all x, see if there is a very high correlation, usually meaning time leakage
#is it useful if i do the analysis

#bayes theorem

#data outlier
#inconsistant data entry
#duplicated rows                                            ----------
#missing NaN, ..

#categorical data
#unstructured data
#normalizastion
#parsing dates


#data leakage(imputing makes shit weaker)


In [74]:
import pandas as pd                                             #
import numpy as np
import matplotlib.pyplot as plt
from  sklearn.model_selection import train_test_split        #

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder     #


from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor                  #



from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
data = pd.read_csv('diamonds.csv')
data.head()

In [75]:

print(data['cut'].value_counts())              #check for abnormal data varienceses
print(data['color'].value_counts())
data['clarity'].value_counts()

b'Ideal'        21551
b'Premium'      13791
b'Very Good'    12082
b'Good'          4906
b'Fair'          1610
Name: cut, dtype: int64
b'G'    11292
b'E'     9797
b'F'     9542
b'H'     8304
b'D'     6775
b'I'     5422
b'J'     2808
Name: color, dtype: int64


b'SI1'     13065
b'VS2'     12258
b'SI2'      9194
b'VS1'      8171
b'VVS2'     5066
b'VVS1'     3655
b'IF'       1790
b'I1'        741
Name: clarity, dtype: int64

In [76]:
print(data.columns)
data.dtypes

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', ''x'',
       ''y'', ''z''],
      dtype='object')


carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price      float64
'x'        float64
'y'        float64
'z'        float64
dtype: object

In [77]:
#for balls2 in data.columns:
    #digma = data[balls2].value_counts()
    #print(digma)


In [78]:
pd.DataFrame(data)

data = data.drop_duplicates(keep = 'first', subset = ['carat','cut','color','clarity','price'])           #drop duplicates
datacopy = data.copy()
xer = data.drop(['price'],axis  = 1)                                                                        #make x
yer = data.price                                                                                               #make y


X_train, X_test, y_train, y_test = train_test_split(xer, yer, test_size=0.2, random_state=1)                            #test splits

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


categorical_cols = [c for c in X_train.columns if X_train[c].nunique() < 20 and X_train[c].dtype == 'object']             #drop over categorical

numerical_cols = [c1 for c1 in X_train.columns if X_train[c1].dtype in ['float', 'int64']]


columns1 = categorical_cols + numerical_cols
X_train = X_train[columns1]

X_test = X_test[columns1]

X_val = X_val[columns1]

In [79]:

categorical_prep = Pipeline(steps=[                                                                         #categorical prepeocessing
    ('Simpleimputer',SimpleImputer(strategy='most_frequent')),                                             #data processing ,Feature imputing
    ('OneHotEncoder1',OneHotEncoder(handle_unknown='ignore'))                                               #data processing ,Feature encoding
])

numerical_prep = SimpleImputer(strategy='constant')                                          #numerical preprocessing #data processing ,Feature imputing

preprocesses = ColumnTransformer(                                                                
    transformers=[
    ('num',numerical_prep,numerical_cols),
    ('cat',categorical_prep,categorical_cols)


])

In [80]:
model = XGBRegressor(n_estimators = 500,learning_rate=0.05, n_jobs=4)                                 #train model,supervised algorithm,xgb

thepipes = Pipeline(steps=[
    ('preprocesses',preprocesses),
    ('model',model)
])

X_test = preprocesses.fit_transform(X_test)                               #preformat eval_set, to have same columns 


thepipes.fit(X_train, y_train, 
model__early_stopping_rounds=5, 
model__eval_set=[(X_test, y_test)],
model__verbose=False )

predictions = thepipes.predict(X_val)
print(mean_absolute_percentage_error(predictions,y_val))


c:\Python39\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.08042829173595333
